In [1]:
import pandas as pd
import numpy as np
import config
from preprocess import preprocess
import modelerWG

In [2]:
df_name = "train_subset_2.csv"
df, colnames = preprocess(df_name)

In [3]:
df.dropna(subset="x1", inplace=True)
df.columns

Index(['ego_id', 'u', 'v', 't', 'x1', 'x2', 'x3', 'city_dist',
       'university_dist', 'sex', 'school_dist', 'age_dist', 'age_mean',
       'school', 'sum', 'u_v', 'gen_0', 'gen_1', 'sex2_0', 'sex2_1', 'sex2_2',
       'city_0', 'city_1', 'city_2', 'university_0', 'university_1',
       'university_2'],
      dtype='object')

In [4]:
rgrs = modelerWG.fit(df, colnames=colnames)

In [4]:
# # импорт модели
# rgrs = modelerWG.import_p('model2.pkl')

In [7]:
pred=modelerWG.predict(df, rgrs, colnames=colnames)

In [8]:
modelerWG.rmse(df, pred)

1.006358875455635

In [5]:
# экспорт модели
modelerWG.export_p('model_fin2.pkl', rgrs)